<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

# Notebook Set D

---

## 03 - Pulse Shaping
In this notebook, we will cover some basic pulse shaping techniques such as square pulse shaping, sinc and raised cosine pulse shapes, matched filtering and the minimization of Inter Symbol Interference (ISI).

## Table of Contents
* [1. Introduction](#introduction)
* [2. Transmitting Pulses](#pulses)
* [3. Square Pulse Shaping](#squares)
    * [3.1. Averaging at the Receiver](#averaging-at-the-receiver)
* [4. Sinc Function](#sinc)
* [5. Raised Cosine](#rcos)
    * [5.1. Raise Cosine Pulse Shaping](#raised-cosine-pulse-shaping)
* [6. Matched Filtering with Root Raised Cosine](#rrc)
* [7. Conclusion](#conclusion)

## Revision
* **v1.0** | 05/12/22 | *First Revision*

---


## 1. Introduction <a class="anchor" id="introduction"></a>

Pulse shaping is an integral part of many communication systems. It serves as a means to band-limit the baseband transmit signal and to improve robustness to noise. Usually, pulse shaping is accomplished by passing our symbols (pulses) through a pulse shaping filter.

You will only need the Numpy and Matplotlib libraries to run this notebook. Begin by importing the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 2. Transmitting Pulses <a class="anchor" id="pulses"></a>
Previously in the baseband modulation notebook, we simulated BPSK (Binary Phase Shift Keying) transmissions, where each symbol was a pulse and represented a bit. However, if we try to transmit a pulse like that over the air we would cause a lot of problems to surrounding radio devices through interference - a pulse (or in this case an impulse) in the time domain actually contains all frequencies in the frequency domain.

We will use the functions defined in previous notebooks to generate some BPSK symbols.

In [ ]:
# Function to add white gaussian noise to a signal, specify SNR in dB
def awgn(signal, snr):
    sig_power = np.mean(np.abs(signal)**2) # calculate signal power
    sig_power_db = 10* np.log10(sig_power) # convert to dB

    noise_power_db = sig_power_db - snr
    noise_power = 10**(noise_power_db / 10)

    complex_noise = np.sqrt(noise_power/2)*(np.random.randn(len(signal)) + \
                                            np.random.randn(len(signal))*1j)

    return signal + complex_noise

# Function to generate BPSK
def generate_bpsk(num_symbols, noise=50):
    bits = np.random.randint(0,2,num_symbols)
    bpsk_scheme = [1+0j, -1+0j]
    bpsk_symbols = np.array([bpsk_scheme[i] for i in bits])
    
    bpsk_symbols = awgn(bpsk_symbols, noise)
    
    return bpsk_symbols

We will now generate BPSK symbols and plot them in the time domain. BPSK only uses the real channel for binary signalling, so we disregard the imaginary part.

In [ ]:
symbols = generate_bpsk(2048)
plt.stem(symbols.real[:8], use_line_collection=True)
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.title('BPSK Pulses')
plt.show()

While this may seem a straightforward way to transfer bits around, looking at the frequency domain of these pulses we can see that they contain all frequencies. This would be undesirable, because it would cause interference to other channels.

In [ ]:
fs = 128
fft_size = 2048

y_fft = np.fft.fftshift(np.fft.fft(generate_bpsk(fft_size),fft_size))[int(fft_size/2):]
freqs = np.fft.fftshift(np.fft.fftfreq(fft_size,1/fs))[int(fft_size/2):]

plt.plot(freqs, np.abs(y_fft))
plt.xlabel('Frequency, Hz')
plt.ylabel('Magnitude')
plt.title('BPSK Pulses in Frequency Domain')
plt.show()

## 3. Square Pulse Shaping <a class="anchor" id="squares"></a>

One of the simplest forms of pulse shaping is the square pulse shape, where we repeat the same pulse $N$ times per symbol - usually we refer to $N$ as samples per symbol, or sps.

In [ ]:
# Number of symbol repeats
N = 8

# Apply square shape to the BPSK symbols
symbols_square_shaped = np.repeat(symbols,N)

# Plot some of the pulse shaped symbols
plt.stem(symbols_square_shaped.real[:64], use_line_collection=True)
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.show()

While this is better than transmitting pulses, square shapes still produce a lot of spectral emissions due to the sharp transitions in the time domain. If we take a clean square wave as an example, we can see that it is made of a combination of many harmonics in the frequency domain.

In [ ]:
# Create clean square wave
square_wave = np.repeat(np.array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1]),8)

# Take the fft of the square wave and calculate corresponding x-axis frequencies
y_fft = np.fft.fftshift(np.fft.fft(square_wave,fft_size))[int(fft_size/2):]
freqs = np.fft.fftshift(np.fft.fftfreq(fft_size,1/fs))[int(fft_size/2):]

fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].plot(square_wave)
ax[0].set_xlabel('Samples')
ax[0].set_ylabel('Amplitude')
ax[0].set_title('Time Domain')
ax[1].plot(freqs, np.abs(y_fft))
ax[1].set_xlabel('Frequency, Hz')
ax[1].set_title('Frequency Domain')
fig.tight_layout()

In a similar way, the FFT of a square pulse shaped waveform will create many side lobes. These have the potential to interfere with users on adjacent frequencies, once the signal is modulated.

In [ ]:
y_fft = np.fft.fftshift(np.fft.fft(symbols_square_shaped,fft_size))[int(fft_size/2):]
freqs = np.fft.fftshift(np.fft.fftfreq(fft_size,1/fs))[int(fft_size/2):]

plt.plot(freqs, np.abs(y_fft))
plt.xlabel('Frequency, Hz')
plt.ylabel('Magnitude')
plt.title('Square Shaped Pulses in Frequency Domain')
plt.show()

### 3.1. Averaging at the receiver <a class="anchor" id="averaging-at-the-receiver"></a>
A square pulse shape receiver can be implemented using a moving average filter. We use 4 weights and set them to [0.25, 0.25, 0.25, 0.25], as we are oversampling with $N=4$ samples per symbol. We anticipate the filter to output a peak response of 1 once the filter fully overlaps with an incoming square pulse shape of 4 samples. Note that using a square pulse shape filter at the transmitter and a moving average filter at the receiver, we do achieve a form of matched filtering, because both filters share the same square shaped response.

In [ ]:
weights = [0.25, 0.25, 0.25, 0.25]
filtered_symbols = np.convolve(np.repeat(symbols,4).real, weights)

Convolving this filter across our pulse shaped signal will create triangular peaks or 'maximum effect points' that will minimise the chance of incorrect symbol decisions (under noisy conditions).

In [ ]:
plt.stem(filtered_symbols.real[:64], use_line_collection=True)
plt.plot(filtered_symbols.real[:64])
plt.show()

Plotting our filtered signal we can see that the side lobes have been suppressed, and as a result we are experience less interference at the receiver.

In [ ]:
# Calculate the one-sided FFT of the filtered signal
y_fft = np.fft.fftshift(np.fft.fft(filtered_symbols,fft_size))[int(fft_size/2):]
freqs = np.fft.fftshift(np.fft.fftfreq(fft_size,1/fs))[int(fft_size/2):]

# Plot the result
plt.plot(freqs, np.abs(y_fft))
plt.xlabel('Frequency, Hz')
plt.ylabel('Magnitude')
plt.title('Square Shaped Pulses in Frequency Domain')
plt.show()

The spectral properties of this receiver are better than transmitting impulses and consuming unnecessary bandwidth. Square pulse shapes still introduce a lot of interference to nearby frequencies. Surely we can do better!

## 4. Sinc Function <a class="anchor" id="sinc"></a>
The ideal bandlimiting response for transmitting our signal would be a “brick wall” — i.e. perfectly passing frequencies up to the actual signal bandwidth, with zero emissions on adjacent frequencies. This "brick wall" response can be achieved by using a sinc shaped filter in the time domain.

The sinc function is given by

$$
n(t) = \frac{sin(\pi t / T)}{\pi t / T}
$$

A true sinc function is infinite. However, this is not realisable, as we would need an infinitely expensive filter with an infinite number of weights. In practice, a truncated sinc shape filter is used instead. Much like windowing functions, it has outputs that taper off at the ends and smooth out the response in order to reduce the sharp transitions that cause unwanted frequency emissions.

In Numpy we can simply call `np.sinc`.

Lets visualize a Sinc shape filter that is 40 symbol periods long and uses 5 samples per symbol. To realize this filter, we will need 251 weights (40 symbols $\times$ 5 samples per symbol + 1 center weight = 251). We assume a sample period of $T$=1s (sampling rate of 1Hz) - in the next code cell we will omit $T$ because it is just 1. Just keep in mind that it is part of the formula.

In [ ]:
T = 1 # sample period
sps = 5 # symbol period in samples
num_weights = 251
x = np.arange(-int(num_weights/2),int(num_weights/2)+1,1)/sps/T
sinc_weights = np.sinc(x)

plt.plot(x,sinc_weights)
plt.title('Sinc function')
plt.xlabel('Time samples, t')
plt.ylabel('n(t)')
plt.show()

Before inspecting the sinc response in the frequency domain, let's generate another filter that runs for only 4 symbol periods to compare the frequency responses.

In [ ]:
# Create another, shorter sinc filter
# num_weights = 4 symbol periods * 5 samples per symbol + 1
num_weights = 21
x = np.arange(-int(num_weights/2),int(num_weights/2)+1,1)/sps
sinc_weights2 = np.sinc(x)

Visualizing the filter responses in the frequency domain, we can see that the more symbol samples the filter is allowed, the closer we get to the ideal cut-off. Desired frequencies of the main lobe are passed, while any unnecessary emissions are suppressed. This relationship basically indicates that more filter weights in time domain means better spectral properties in frequency domain.

In [ ]:
# Calculate frequency responses of the 2 filters
fft_size = 1024
sinc1_response = 20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(sinc_weights, fft_size)))/fs)
sinc2_response = 20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(sinc_weights2, fft_size)))/fs)

# Plot the frequency responses
plt.plot(sinc1_response[int(fft_size/2):-250]) # truncated for better viewing experience
plt.plot(sinc2_response[int(fft_size/2):-250])

plt.legend(('50 symbol periods', '4 symbol periods'))
plt.title('Sinc filter magnitude responses')
plt.ylabel('Magnitude, Hz')
plt.xlabel('Frequency, Hz')
plt.show()

What if we are limited in the number of filter weights we want to use (which we often are), but still want to minimze the outer frequency emissions? Well we can improve our sinc filter.

## 5. Raised Cosine <a class="anchor" id="rcos"></a>
The Raised Cosine (RC) pulse shape has a preferable impulse response that tails off more quickly (as governed by the design parameter α). It is defined as:

$$
p(t) = (\frac{sin(\pi t / T)}{\pi t / T}) (\frac{cos ( \alpha \pi t / T)}{1 - (2\alpha t / T)^{2}})
$$

Note that if $\alpha = 0$, the second part of the equation will be equal to 1, and we end up with a response equivalent to the sinc function.

Here, we can visualize what effect the parameter $\alpha$ has on our new Raised Cosine filter response. We can see that the cosine window term suppresses the tails of the Sinc shape as the $\alpha$ term increases - this allows us to design a cheaper pulse shaping filter.

In [ ]:
sps = 5
num_weights = 101
x = 0.9999*np.arange(-int(num_weights/2),int(num_weights/2)+1,1)/sps

alphas = [0, 0.25, 0.5, 0.75]
for alpha in alphas:
    raised_cos_weights = np.sinc(x)*(np.cos(alpha*np.pi*x)/(1-((2*alpha*x)**2)))
    plt.plot(x,raised_cos_weights)

plt.legend((r'$\alpha = 0$', r'$\alpha = 0.25$', r'$\alpha = 0.5$', r'$\alpha = 0.75$'))
plt.ylabel('Filter weight magnitude')
plt.xlabel('Filter weight index')
plt.show()

Once we look at the filter responses at different $\alpha$ values, we can see that the required bandwidth to capture our signal increases with $\alpha$. Like most things, the $\alpha$ parameter is a tradeoff between how much spectral leakage you are allowing to nearby frequencies and how much bandwidth will be required to recover the transmitted signal.

In [ ]:
sps = 5
num_weights = 121
x = 0.9999*np.arange(-int(num_weights/2),int(num_weights/2)+1,1)/sps

for alpha in alphas:
    raised_cos_weights = np.sinc(x)*(np.cos(alpha*np.pi*x)/(1-((2*alpha*x)**2)))

    rcos_response = 20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(raised_cos_weights, fft_size)))/sps)
    plt.plot(rcos_response[int(fft_size/2):-100])
    
plt.legend((r'$\alpha = 0$', r'$\alpha = 0.25$', r'$\alpha = 0.5$', r'$\alpha = 0.75$'))
plt.title('Magnitude Response')
plt.xlabel('Frequency, Hz')
plt.ylabel('Magnitude, dB')
plt.show()

### 5.1. Raised Cosine Pulse Shaping <a class="anchor" id="raised-cosine-pulse-shaping"></a>

We will begin by using a simple 8 symbol period, or 41 weight RC filter.

In [ ]:
fs = 5
num_weights = 41
alpha = 0.5
x = 0.9999*np.arange(-int(num_weights/2),int(num_weights/2)+1,1)/fs
raised_cos_weights = np.sinc(x)*(np.cos(alpha*np.pi*x)/(1-((2*alpha*x)**2)))

Here we generate 3 pulses, each 5 samples or 1 symbol period apart, so they do not interfere with each other. Notice how pulse shaping causes a delay in the signal - our first peak is no longer at index 5, but 25! This is because pulse shaping introduces latency, and for our RC filter this is half the total pulse duration, which is 4 symbol periods (or 20 samples).

In [ ]:
# Generate pulse vectors
pulse1 = np.zeros(21,)
pulse2 = np.zeros(21,)
pulse3 = np.zeros(21,)
pulse1[5] = 1
pulse2[10] = 1
pulse3[15] = 1

# Pulse shape each symbol separately 
symbol1_rc = np.convolve(pulse1,raised_cos_weights).real
symbol2_rc = np.convolve(pulse2,raised_cos_weights).real
symbol3_rc = np.convolve(pulse3,raised_cos_weights).real

# Pulse shape the combination of symbols
symbols_rc = np.convolve(pulse1+pulse2+pulse3,raised_cos_weights).real

# Plot them all
fig, ax = plt.subplots(1,3, figsize=(16,4))

ax[0].plot(pulse1, '-o')
ax[0].plot(pulse2, '-o')
ax[0].plot(pulse3, '-o')
ax[0].set_title('Individual Pulses')
ax[0].set_xlabel('Samples')
ax[0].set_ylabel('Amplitude')
ax[0].legend(('Pulse 1', 'Pulse 2', 'Pulse 3'), loc='center left')

ax[1].plot(symbol1_rc)
ax[1].plot(symbol2_rc)
ax[1].plot(symbol3_rc)
ax[1].plot([15,20,25,30,35,40,45], [0,0,0,0,0,0,0], 'x')
ax[1].set_title('RC Pulse Shaped Symbols')
ax[1].set_xlabel('Samples')
ax[1].set_ylabel('Amplitude')
ax[1].legend(('Symbol 1', 'Symbol 2', 'Symbol 3', 'Zero Crossings'), loc='center left')

ax[2].plot(symbols_rc)
ax[2].plot([5+20,10+20,15+20], [1,1,1],'x')
ax[2].set_title('RC Pulse Shaped Transmission')
ax[2].set_xlabel('Samples')
ax[2].set_ylabel('Amplitude')
ax[2].legend(('Transmission', 'Maximum Effect Points'), loc='center left')
fig.tight_layout()

The great thing about the RC filter is that it does not introduce Inter Symbol Interference (ISI) - that is the maximum effect points are unaffected by the neighbouring symbols. While they do overlap with each other, their amplitude is exactly 0 at the peaks of other symbols.

## 6. Matched Filtering with Root Raised Cosine <a class="anchor" id="rrc"></a>

In order to reap the benefits of the raised cosine with matched filtering, we split it into two filters with identical responses. The RC shape is split into two, resulting in two (identical, or “matched”) Root Raised Cosine (RRC) filters. The RRC filter is one of the most popular matched filtering options in many communications systems. The mathematics of taking a square root in the frequency domain are fairly complicated. It is not a trivial filter to implement in the time domain, as can be seen by the impulse response definition in the equation below.

$$
h(t) =
    \begin{cases}
      \frac{1}{T_s}(1+\alpha(\frac{4}{\pi}-1)), & \text{if } t=0,\\
      \frac{\alpha}{T_{s}\sqrt{2}}[(1+\frac{2}{\pi})sin(\frac{\pi}{4\alpha}) + (1 - \frac{2}{\pi})cos(\frac{\pi}{4\alpha})] & \text{if } t=\pm\frac{T_s}{4\alpha},\\
      \frac{1}{T_s}\frac{sin[\pi\frac{t}{Ts}(1-\alpha)]+4\alpha\frac{t}{T_s}cos[\pi\frac{t}{T_s}(1+\alpha)]}{\pi\frac{t}{T_s}[1-(4\alpha\frac{t}{T_s})^2]} & \text{otherwise.}
    \end{cases}
$$
    
Luckily most filter design tools come equipped with RRC implementations, due to it being such a popular choice for matched filtering. If you were using the filter design tools like those found in GNU Radio or MATLAB, you need not worry about these types of equations. Here, we will implement a simplified version, by making a small filter following the 3rd part of this equation, which should be enough to demonstrate the effectiveness of this type of filtering. We will see a division error, because we are applying our formula to a value that is undefined. We compensate for this by defining our middle filter weight as shown in the 1st part of the equation. The term $\frac{t}{T_s}$ in our code cell is defined as $x$.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Base parameters, same as rcos in the previous example
fs = 5
Ts =  1/fs
alpha = 0.5

num_weights = 41
x = np.arange(-int(num_weights/2),int(num_weights/2)+1,1)/fs

# Impulse response according to definition
h_rrc = 1/Ts*(np.sin(np.pi*x*(1-alpha)) + 4*alpha*x*np.cos(np.pi*x*(1+alpha)))/(np.pi*x*(1-(4*alpha*x)**2))

# Find and replace the center weight according to the first part of the formula
h_rrc[int(num_weights/2)] = 1/Ts*(1+alpha*(4/np.pi - 1))

# Normalize the weights
h_rrc = h_rrc/np.max(h_rrc)

# Plot filter impulse response
plt.plot(h_rrc)
plt.title('Raised Root Cosine Impulse Response')
plt.xlabel('Time samples, t')
plt.ylabel('Filter response, h(t)')

Unlike the raised cosine, the RRC does introduce ISI to our transmitted pulses, which is why it is so important to pair it with the same filter on the receiver. If we pass our pulses through this filter a second time, the zero-ISI property is restored and we can see that the maximum effect points match the zero crossings.

In [ ]:
# Generate pulse vectors
pulse1 = np.zeros(21,)
pulse2 = np.zeros(21,)
pulse3 = np.zeros(21,)
pulse1[5] = 1
pulse2[10] = 1
pulse3[15] = 1

# Pulse shape each symbol separately 
symbol1_rrc = np.convolve(pulse1,h_rrc).real
symbol2_rrc = np.convolve(pulse2,h_rrc).real
symbol3_rrc = np.convolve(pulse3,h_rrc).real

# Pulse shape with matched filter
symbol1_matched = np.convolve(symbol1_rrc,h_rrc).real[20:-20]
symbol2_matched = np.convolve(symbol2_rrc,h_rrc).real[20:-20]
symbol3_matched = np.convolve(symbol3_rrc,h_rrc).real[20:-20]

# Plot them all
fig, ax = plt.subplots(1,2, figsize=(12,4))

# Only RRC filtered pulses
ax[0].plot(symbol1_rrc)
ax[0].plot(symbol2_rrc)
ax[0].plot(symbol3_rrc)
ax[0].plot([15,20,25,30,35,40,45], [0,0,0,0,0,0,0], 'x')
ax[0].set_title('RRC Pulse Shaped Symbols')
ax[0].set_ylabel('Magnitude')
ax[0].set_xlabel('Samples')
ax[0].legend(('Symbol 1', 'Symbol 2', 'Symbol 3', 'Zero Crossings'), loc='center left')

# Twice RRC filtered pulses, demonstrating matched filtering
ax[1].plot(symbol1_matched)
ax[1].plot(symbol2_matched)
ax[1].plot(symbol3_matched)
ax[1].plot([15,20,25,30,35,40,45], [0,0,0,0,0,0,0], 'x')
ax[1].set_title('After Matched Filtering')
ax[1].set_ylabel('Magnitude')
ax[1].set_xlabel('Samples')
ax[1].legend(('Symbol 1', 'Symbol 2', 'Symbol 3', 'Zero Crossings'), loc='center left')
plt.tight_layout()

## 7. Conclusion

In this notebook we covered some pulse shaping essentials, including the sinc and raised cosine pulse shapes. We showcased the tradeoffs of filter size and desired frequency response, as well as how the $\alpha$ parameter for the raised cosine filter can allow us to tune some spectral properties of the pulse shaping filter without changing the number of available filter weights. The next set of notebooks will discuss amplitude modulation.

---

[⬅️ Previous Notebook](02_evm_and_ber.ipynb) || [Next Notebook 🚀](../notebook_E/01_amplitude_modulation.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---